## NA imputation 

In [1]:
import jusipy
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import missingno as msno
import copy
#from fast_imputation import fast_imputation

#### Loading...

In [2]:
all_points = pd.read_pickle('data/all_points.pkl')

In [3]:
all_points.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22107 entries, 0 to 19999
Data columns (total 6 columns):
lat         22107 non-null object
long        22107 non-null float64
country     22107 non-null object
year        22107 non-null object
positive    22107 non-null int64
random      22107 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.2+ MB


In [4]:
A_country = jusipy.country_features.All()

Loading WorldBank(/home/marios/Desktop/Personal/repos/JuSiPy_Asser/jusipy/country_features/data/SD_API_15_DS2_en_csv_v2_10186395.csv)                                                                                                    

In [5]:
all_country_features = jusipy.country_features.get(all_points, A_country, fuzzy=50)

In [6]:
fields = ['WB_ARD__AG_SRF_TOTL_K2', 'WB_SE__WB_NY_GDP_MKTP_KD_ZG', 'WB_ARD__SP_RUR_TOTL', 'WB_ARD__EG_ELC_ACCS_RU_ZS', 'WB_ARD__AG_LND_TOTL_RU_K2',
          'WB_ARD__AG_LND_FRST_K2', 'WB_ARD__AG_LND_ARBL_ZS', 'WB_ARD__AG_LND_AGRI_K2', 'WB_ARD__AG_CON_FERT_ZS','WB_ARD__AG_LND_PRCP_MM','WB_ARD__EN_ATM_METH_AG_KT_CE',
         'WB_ARD__EN_ATM_NOXE_AG_KT_CE','WB_ARD__NV_AGR_TOTL_ZS','WB_ARD__TM_VAL_AGRI_ZS_UN','WB_ARD__TX_VAL_AGRI_ZS_UN','WB_ARD__AG_LND_TOTL_K2',
         'WB_ARD__SL_AGR_EMPL_ZS','WB_SPL__SL_TLF_0714_ZS', 'TI_CPI__TI_CPI_CPISN']

In [13]:
X = all_country_features

In [31]:
Y = X.drop_duplicates()

In [32]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 769 entries, 0 to 14836
Columns: 408 entries, WB_LGAF__WB_LGAF2016_1_1_1 to UN_HDI__Gini_coefficient
dtypes: float64(280), object(128)
memory usage: 2.4+ MB


In [103]:
Z = Y[fields]

In [104]:
Z[Z.isna().all(axis=1)]

,WB_ARD__AG_SRF_TOTL_K2,WB_SE__WB_NY_GDP_MKTP_KD_ZG,WB_ARD__SP_RUR_TOTL,WB_ARD__EG_ELC_ACCS_RU_ZS,WB_ARD__AG_LND_TOTL_RU_K2,WB_ARD__AG_LND_FRST_K2,WB_ARD__AG_LND_ARBL_ZS,WB_ARD__AG_LND_AGRI_K2,WB_ARD__AG_CON_FERT_ZS,WB_ARD__AG_LND_PRCP_MM,WB_ARD__EN_ATM_METH_AG_KT_CE,WB_ARD__EN_ATM_NOXE_AG_KT_CE,WB_ARD__NV_AGR_TOTL_ZS,WB_ARD__TM_VAL_AGRI_ZS_UN,WB_ARD__TX_VAL_AGRI_ZS_UN,WB_ARD__AG_LND_TOTL_K2,WB_ARD__SL_AGR_EMPL_ZS,WB_SPL__SL_TLF_0714_ZS,TI_CPI__TI_CPI_CPISN
index,,,,,,,,,,,,,,,,,,,
1941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
Z2 = Z.drop([1941,189,1469])

In [106]:
Im = jusipy.impute.MiceImputer(seed_strategy='median')

ValueError: Found array with 0 feature(s) (shape=(765, 0)) while a minimum of 1 is required.

In [175]:
import numpy as np
import sklearn
from sklearn.preprocessing import Imputer
import pandas as pd
def fit(X):      
    x = X.fillna(value=np.nan)

    null_check = x.isnull().any()
    null_data = x[null_check.index[null_check]]

    for i in null_data:
        y = null_data[i]
        y_notnull = y[y.notnull()]

        model_list = []
        if True:
            imp = Imputer(strategy='median')
            model_list.append(imp.fit(x))
            non_null_data = pd.DataFrame(imp.fit_transform(x))

        else:
            non_null_data = x[null_check.index[~null_check]]


        x_notnull = non_null_data[y.notnull()]
        return x_notnull

#         if y.nunique() > 2:
#             model = LinearRegression()
#             model.fit(x_notnull, y_notnull)
#             model_list.insert(0, model)
#             self.model_dict_.update({i: model_list})
#         else:
#             model = LogisticRegression()
#             model.fit(x_notnull, y_notnull)
#             model_list.insert(0, model)
#             self.model_dict_.update({i: model_list})


-------------------------------------------------------------------------------

In [176]:
fit(Z2)

/home/marios/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/marios/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


ValueError: cannot reindex from a duplicate axis

### NA percentage (in red)

In [110]:
# somehow unconsciouly it came out like the French flag..

In [111]:
fields = ['WB_ARD__AG_SRF_TOTL_K2', 'WB_SE__WB_NY_GDP_MKTP_KD_ZG', 'WB_ARD__SP_RUR_TOTL', 'WB_ARD__EG_ELC_ACCS_RU_ZS', 'WB_ARD__AG_LND_TOTL_RU_K2',
          'WB_ARD__AG_LND_FRST_K2', 'WB_ARD__AG_LND_ARBL_ZS', 'WB_ARD__AG_LND_AGRI_K2', 'WB_ARD__AG_CON_FERT_ZS','WB_ARD__AG_LND_PRCP_MM','WB_ARD__EN_ATM_METH_AG_KT_CE',
         'WB_ARD__EN_ATM_NOXE_AG_KT_CE','WB_ARD__NV_AGR_TOTL_ZS','WB_ARD__TM_VAL_AGRI_ZS_UN','WB_ARD__TX_VAL_AGRI_ZS_UN','WB_ARD__AG_LND_TOTL_K2',
         'WB_ARD__SL_AGR_EMPL_ZS','WB_SPL__SL_TLF_0714_ZS', 'TI_CPI__TI_CPI_CPISN']
fields_3 = list(country_features.keys())
#for x in fields_3:
#    df = country_features[x]
#    if x in fields:
#        print('\033[94m'+x+': '+'\033[92m'+A_country.description[x]+' has '+str(sum(df.isna()))+' NA '+'\033[91m'+str(sum(df.isna())/len(df)))
#    else:
#        print('\033[94m'+x+': '+'\033[0m'+A_country.description[x]+' has '+str(sum(df.isna()))+' NA '+'\033[91m'+str(sum(df.isna())/len(df)))
nas = []
nas_per = []
des = []
for x in fields_3:
    df = country_features[x]
    nas.append(sum(df.isna()))
    nas_per.append(sum(df.isna())/len(df))
    des.append(A_country.description[x])

    ###Printing the ones with NA < 10%
# DF = pd.DataFrame({'Indicator': fields_3, 'Description': des, 'Percentage of NA':nas_per})
# DF[DF["Percentage of NA"]<0.1]

# def color_field(val):
 
#     color = 'green' if val in fields else None
#     return 'color: %s' % color

# s = DF.style.applymap(color_field)
# s

NameError: name 'country_features' is not defined

In [112]:
fields = ['WB_ARD__AG_SRF_TOTL_K2', 'WB_SE__WB_NY_GDP_MKTP_KD_ZG', 'WB_ARD__SP_RUR_TOTL', 'WB_ARD__EG_ELC_ACCS_RU_ZS', 'WB_ARD__AG_LND_TOTL_RU_K2',
          'WB_ARD__AG_LND_FRST_K2', 'WB_ARD__AG_LND_ARBL_ZS', 'WB_ARD__AG_LND_AGRI_K2', 'WB_ARD__AG_CON_FERT_ZS','WB_ARD__AG_LND_PRCP_MM','WB_ARD__EN_ATM_METH_AG_KT_CE',
         'WB_ARD__EN_ATM_NOXE_AG_KT_CE','WB_ARD__NV_AGR_TOTL_ZS','WB_ARD__TM_VAL_AGRI_ZS_UN','WB_ARD__TX_VAL_AGRI_ZS_UN','WB_ARD__AG_LND_TOTL_K2',
         'WB_ARD__SL_AGR_EMPL_ZS']
imputed_ = []
for x in fields:
    df_temp = country_features[x]
    imputed_.append(fast_imputation(df_temp))


NameError: name 'country_features' is not defined